In [1]:
# API headers
# https://stackoverflow.com/questions/46862719/pythons-requests-library-timing-out-but-getting-the-response-from-the-browser

In [2]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
print('Number of teams fetched: {}'.format(len(nba_teams)))

df = pd.DataFrame.from_dict(nba_teams)
df

Number of teams fetched: 30


,abbreviation,city,full_name,id,nickname,state,year_founded
0,ATL,Atlanta,Atlanta Hawks,1610612737,Hawks,Atlanta,1949
1,BOS,Boston,Boston Celtics,1610612738,Celtics,Massachusetts,1946
2,CLE,Cleveland,Cleveland Cavaliers,1610612739,Cavaliers,Ohio,1970
3,NOP,New Orleans,New Orleans Pelicans,1610612740,Pelicans,Louisiana,2002
4,CHI,Chicago,Chicago Bulls,1610612741,Bulls,Illinois,1966
5,DAL,Dallas,Dallas Mavericks,1610612742,Mavericks,Texas,1980
6,DEN,Denver,Denver Nuggets,1610612743,Nuggets,Colorado,1976
7,GSW,Golden State,Golden State Warriors,1610612744,Warriors,California,1946
8,HOU,Houston,Houston Rockets,1610612745,Rockets,Texas,1967
9,LAC,Los Angeles,Los Angeles Clippers,1610612746,Clippers,California,1970


## Pseudo code

1. Get team IDs for Raptors and Golden State
2. Get all the players on Raptors
    * Might have to get only the Raptors that played in playoffs against GS
3. For each player, get their passes received and passes made
4. Create dataframe with all passes

## Step 1: Get team IDs for Toronto Raptors and Golden State Warriors

In [4]:
team_ids = teams.find_teams_by_full_name('^(Golden State|Toronto)')
team_ids

[{'id': 1610612744,
  'full_name': 'Golden State Warriors',
  'abbreviation': 'GSW',
  'nickname': 'Warriors',
  'city': 'Golden State',
  'state': 'California',
  'year_founded': 1946},
 {'id': 1610612761,
  'full_name': 'Toronto Raptors',
  'abbreviation': 'TOR',
  'nickname': 'Raptors',
  'city': 'Toronto',
  'state': 'Ontario',
  'year_founded': 1995}]

## Step 2: Get all the players on the Raptors in 2018-19 season

In [7]:
custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

# use the CommonTeamRoster endpoint
from nba_api.stats.endpoints import commonteamroster

season='2018-19'
team_id='1610612761'

tor_players = commonteamroster.CommonTeamRoster(season=season, team_id=team_id, headers=custom_headers, timeout=300)
tor_players = tor_players.get_data_frames()[0]

player_ids=tor_players['PLAYER_ID'].tolist()
# for player in tor_players.get_data_frames()[0].itertuples():
#     print(player.PLAYER_ID)

# for player_id in player_ids:
#     print(player_id)

# player_id_sample = player_ids[1:5]
# player_id_sample
player_ids

[1627775,
 202695,
 1628384,
 200768,
 1628070,
 201586,
 1626259,
 201980,
 203961,
 202391,
 201975,
 1627832,
 1626181,
 1628449,
 201188,
 1627783]

## Step 3: For each player, get their passes received and passes made

In [8]:
# https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playerdashptpass.md
from nba_api.stats.endpoints import playerdashptpass

# set up parameters

# required (league_id default is '00' which is NBA)
last_n_games='0'
month='0'
opponent_team_id='1610612744'
per_mode_simple='PerGame'
#player_id='1627775'
season='2018-19'
team_id='0'
season_type_all_star='Playoffs'

In [20]:
appended_data = []

for player_id in player_ids:
    passes_received_temp = playerdashptpass.PlayerDashPtPass(
        last_n_games=last_n_games,
        month=month,
        opponent_team_id=opponent_team_id,
        per_mode_simple=per_mode_simple,
        player_id=str(player_id),
        season=season,
        team_id=team_id,
        season_type_all_star=season_type_all_star,
        headers=custom_headers, timeout=100
    )

    passes_received_df_temp = passes_received_temp.get_data_frames()[1]
    appended_data.append(passes_received_df_temp)

In [ ]:
for player_id in player_ids[0:4]:
    print(player_id)

# Ideas

1. Gantt chart of team founded date https://plot.ly/python/gantt/
2. Something about higher scoring conditional on who passed to whom
3. Shot charts for fun http://savvastjortjoglou.com/nba-shot-sharts.html

In [ ]:
len(player_ids)

In [21]:
passes_received_df_temp

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_FROM,PASS_TEAMMATE_PLAYER_ID,FREQUENCY,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"VanVleet, Fred",1627832,0.185,...,0.67,1.33,3.50,0.381,1.33,2.83,0.471,0.00,0.67,0.000
1,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Powell, Norman",1626181,0.034,...,0.17,0.33,0.33,1.000,0.33,0.33,1.000,0.00,0.00,NaN
2,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Leonard, Kawhi",202695,0.182,...,0.83,0.83,2.83,0.294,0.67,2.00,0.333,0.17,0.83,0.200
3,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Lin, Jeremy",202391,0.003,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
4,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Green, Danny",201980,0.068,...,0.33,0.67,0.83,0.800,0.67,0.83,0.800,0.00,0.00,NaN
5,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Ibaka, Serge",201586,0.045,...,0.17,0.17,0.33,0.500,0.00,0.00,NaN,0.17,0.33,0.500
6,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Gasol, Marc",201188,0.168,...,0.17,0.83,1.83,0.455,0.83,1.67,0.500,0.00,0.17,0.000
7,1627783,"Siakam, Pascal",Toronto Raptors,1610612761,TOR,received,6,"Lowry, Kyle",200768,0.315,...,2.50,3.00,5.50,0.545,2.50,4.17,0.600,0.50,1.33,0.375


In [22]:
appended_data
appended_data = pd.concat(appended_data)

In [23]:
appended_data

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_FROM,PASS_TEAMMATE_PLAYER_ID,FREQUENCY,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Siakam, Pascal",1627783,0.077,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
1,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Leonard, Kawhi",202695,0.538,...,0.25,0.25,0.25,1,0.00,0.00,NaN,0.25,0.25,1
2,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Green, Danny",201980,0.154,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
3,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Ibaka, Serge",201586,0.077,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
4,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Gasol, Marc",201188,0.077,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
5,1627775,"McCaw, Patrick",Toronto Raptors,1610612761,TOR,received,4,"Lowry, Kyle",200768,0.077,...,0.00,0.00,0.25,0,0.00,0.25,0,0.00,0.00,NaN
0,202695,"Leonard, Kawhi",Toronto Raptors,1610612761,TOR,received,6,"VanVleet, Fred",1627832,0.137,...,0.33,0.83,2.33,0.357,0.67,1.33,0.5,0.17,1.00,0.167
1,202695,"Leonard, Kawhi",Toronto Raptors,1610612761,TOR,received,6,"Siakam, Pascal",1627783,0.173,...,1.33,2.67,3.67,0.727,1.67,2.50,0.667,1.00,1.17,0.857
2,202695,"Leonard, Kawhi",Toronto Raptors,1610612761,TOR,received,6,"McCaw, Patrick",1627775,0.010,...,0.00,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN
3,202695,"Leonard, Kawhi",Toronto Raptors,1610612761,TOR,received,6,"Powell, Norman",1626181,0.029,...,0.00,0.00,0.33,0,0.00,0.17,0,0.00,0.17,0


In [26]:
appended_data.to_csv('tor_passes_received.csv', index=False)